<a href="https://colab.research.google.com/github/kariya-v2/Portfolio/blob/main/%E8%AA%B2%E9%A1%8C04_11_KS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **プログラムの名称**
##古典暗号
## **概要**
シーザー暗号、単一換字式暗号、エニグマ暗号を模したプログラムです。暗号化したい文を入れると暗号化されて出てきます。暗号化はアルファベットのみ対応してます。


## **操作方法**
操作方法は暗号化したいものを入力するだけです。シーザー暗号の場合はずらしたい分の数字を入力もしてください。エニグマ暗号の場合なぜか日本語も暗号化されるので注意してください。

## **制作者のコメント**
基本的にネットやGPTを参考にせざるを得なかったので自分の要素がほとんどないです。でも、pythonの理解度が格段に上がったので気分は上々です。次に同じような課題などがあればできるだけ自身の知識だけを頼りにプログラミングしたい所存です。
また、今回作成したプログラムはアルファベットだけじゃなくて日本語も対応させてみたかったです。もっと早くに取り掛かってればテストが来る前に理想の完成形に近づいてたんだろうなと思いました。

## **作成時間**

作成時間：**約17時間**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###シーザー暗号

In [ ]:
alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' #アルファベットの集合
def caesar_cipher_encrypt(text, shift):
    encrypted_text = ""      #暗号化したものを入れる変数
    for char in text:        #一文字ずつ暗号化
        if char.isalpha(): #A～Zの大文字のアルファベットであるかを判別
            char = char.upper()
            encrypted_char = chr((ord(char) - ord('A') + shift) % 26 + ord('A')) #文字をユニコードポイントに変換し、ずらしたい数字を足して、文字に変換
            encrypted_text += encrypted_char      #暗号化したものを足す
        else:
            encrypted_text += "text" #アルファベット以外はそのまま出力
    return encrypted_text

#入力部分
hirabun = input("暗号化したい文を入力 (アルファベットのみ) : ")
shift = int(input("ずらしたい数字を入力 : "))

#出力部分
encrypted_text = caesar_cipher_encrypt(hirabun, shift)
print(f"暗号文 -{encrypted_text}-")

暗号化したい文を入力 (アルファベットのみ) : aaasasas
ずらしたい数字を入力 : 3
暗号文 -DDDVDVDV-


##単一換字式暗号

In [ ]:
import random

def generate_substitution_cipher():
    shuffled = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ') #アルファベットのリストの作成
    random.shuffle(shuffled) #リストをシャッフル
    # 辞書を作成
    cipher_dict = {alphabet[i]: shuffled[i] for i in range(len(alphabet))}
    return cipher_dict

def encrypt(message, cipher_dict):
    encrypted_message = ''   #暗号文を入れる変数
    for char in message:
        if char.upper() in cipher_dict:  #アルファベットであるかを判別
            # 大文字のアルファベットを対応表から変換
            if char.isupper():
                encrypted_message += cipher_dict[char]
            # 小文字のアルファベットを対応表から変換
            else:
                encrypted_message += cipher_dict[char.upper()].lower()
        else:
            encrypted_message += char  # アルファベット以外はそのまま出力
    return encrypted_message

#入力部分
message = input("暗号化したい文を入力 (アルファベットのみ) : ")

#出力部分
cipher_dict = generate_substitution_cipher()
print("対応表:", cipher_dict)

encrypted_message = encrypt(message, cipher_dict)
print(f"暗号文 -{encrypted_message}-")


暗号化したい文を入力 (アルファベットのみ) : aaddw333
対応表: {'A': 'X', 'B': 'N', 'C': 'O', 'D': 'F', 'E': 'E', 'F': 'Q', 'G': 'G', 'H': 'B', 'I': 'A', 'J': 'C', 'K': 'P', 'L': 'W', 'M': 'V', 'N': 'H', 'O': 'U', 'P': 'D', 'Q': 'T', 'R': 'I', 'S': 'L', 'T': 'Z', 'U': 'M', 'V': 'Y', 'W': 'R', 'X': 'K', 'Y': 'S', 'Z': 'J'}
暗号文 -xxffr333-


##エニグマ暗号

In [ ]:

#ローター(文字変換するところ)のクラス
class Rotor:
    def __init__(self, wiring, notch):
        self.wiring = wiring
        self.notch = notch
        self.position = 0

    #初期位置の設定
    def set_position(self, position):
        self.position = position

    #ローターの回転
    def rotate(self):
        self.position = (self.position + 1) % 26

    #文字変換する部分
    def encrypt(self, letter):
        index = (ord(letter) - ord('A') + self.position) % 26  # ローターの位置を考慮して文字を数値に変換
        encrypted_letter = self.wiring[index]
        return chr((ord(encrypted_letter) - ord('A') - self.position) % 26 + ord('A'))

    #今のローターが回転して特定の文字に達すると隣のローターが回転する部分
    def at_notch(self):
        return self.wiring[self.position] == self.notch

#リフレクター(折り返すところ)のクラス
class Reflector:
    def __init__(self, wiring):
        self.wiring = wiring

    #文字変換する部分
    def reflect(self, letter):
        index = ord(letter.upper()) - ord('A')
        if 0 <= index < len(self.wiring):  #アルファベットであるか判断する
            return self.wiring[index]
        else:
            return letter

#プラグボード(最初に対になる文字を入れ替えるところ)のクラス
class Plugboard:
    def __init__(self, connections):
        self.connections = connections
    #文字を入れ替える部分
    def swap(self, letter):
        return self.connections.get(letter, letter)

#エニグマのクラス
class EnigmaMachine:
    def __init__(self, rotors, reflector, plugboard):
        self.rotors = rotors
        self.reflector = reflector
        self.plugboard = plugboard

    #ローターの位置を設定する部分
    def set_rotor_positions(self, positions):
        for rotor, position in zip(self.rotors, positions):
            rotor.set_position(position)

    #暗号化部分
    def encrypt(self, message):
        encrypted_message = []
        if message.isalpha():

            if letter.isalpha():  # アルファベットのみ処理
                letter = letter.upper()

                # プラグボードを通過
                letter = self.plugboard.swap(letter)

                # 各ローターを通過させる
                for rotor in self.rotors:
                    letter = rotor.encrypt(letter)

                # リフレクターを通過
                letter = self.reflector.reflect(letter)

                # 各ローターを逆に通過させる
                for rotor in reversed(self.rotors):
                    letter = rotor.encrypt(letter)

                encrypted_message.append(letter)  # 暗号化された文字をリストに追加

                # ローターを回転させる
                for i in range(len(self.rotors)):
                    self.rotors[i].rotate()
                    if not self.rotors[i].at_notch(): #一つのローターだけが回転
                        break
            else:
                encrypted_message.append(letter)  # アルファベット以外はそのまま出力
        return ''.join(encrypted_message)

# ローターの配線とリフレクターの設定(お好みのアルファベットの順番)
rotor1 = Rotor("EKMFLGDQVZNTOWYHXUSPAIBRCJ", 'Q') #この設定なら左の文字列がローターの配線部分、Qまで回転するたびにrotor2が一回回転する
rotor2 = Rotor("AJDKSIRUXBLHWTMCQGZNPYFVOE", 'E')
rotor3 = Rotor("BDFHJLCPRTXVZNYEIWGAKMUSQO", 'V')
reflector = Reflector("YRUHQSLDPXNGOKIETZJWVFMCB")

# プラグボードの設定(好きなように追加してもよい)
plugboard_connections = {
    'A': 'B',
    'B': 'A',
    'C': 'D',
    'D': 'C'
}
plugboard = Plugboard(plugboard_connections)

# エニグマ機の初期化
enigma = EnigmaMachine([rotor1, rotor2, rotor3], reflector, plugboard)

# ローターの初期位置を設定
initial_positions = [0, 0, 0]
enigma.set_rotor_positions(initial_positions)

# メッセージの暗号化
message = input("暗号化したい文を入力 (アルファベットのみ) : ")
encrypted_message = enigma.encrypt(message)
print(f"暗号文 - {encrypted_message} -")

暗号化したい文を入力 (アルファベットのみ) : aAあ
暗号文 - QLQ -
